In [1]:
import spacy
import corpus_tools
import corruption_tools 

FileNotFoundError: [Errno 2] No such file or directory: 'lemario_snowball.txt'

In [2]:
nlp_small = spacy.load('es_core_news_sm')
nlp_med = spacy.load('es_core_news_md')

In [40]:
print(len (nlp_small.vocab))
print(len (nlp_med.vocab))

37940
1232137


In [36]:
import os
# Load the inflected verb form list
verb_list = []
verb_list_fn = os.path.abspath("../resources/verbos-espanol-conjugaciones.txt")
with open(verb_list_fn) as verb_file:
    for line in verb_file.readlines():
        verb_list.append(line.strip())

In [39]:
verbs_in = 0
verbs_not_in = 0
for verb in verb_list:
    verb_id = nlp_med.vocab.strings[verb]
    if verb_id in nlp_med.vocab:
        verbs_in += 1
    else:
        verbs_not_in += 1
print(verbs_in)
print(verbs_not_in)

68926
474644


In [3]:
print( len(nlp_small.vocab))
print(len(nlp_med.vocab))

# Initialize the array of corruptors
corruptortypes = ["verbRM", "verbInfl", "adjInfl"]
corruptors = {}
corruptors["verbRM"] = transformer.VerbRemover("verbRM")
corruptors["verbInfl"] = transformer.VerbInflCorruptor("verbInfl")
corruptors["adjInfl"] = transformer.AdjInflCorruptor("adjInfl")
# Initialize counters for corrupted sentences
corruptCount = {}
uncorrupted_count = 0
for typ in corruptortypes:
    corruptCount[typ] = 0

# Load sentence generator
# in_corpus_filename = sys.argv[1]()
in_corpus_filename = os.path.abspath("../data/exp1_mini/exp1_mini-CT")
out_corpus_folder = in_corpus_filename + "."
in_corpus_file = open(in_corpus_filename, "r")
sentence_gen = sentence_generator(in_corpus_file, nlp_med)

# Create outfiles for each type of corrupted sentence
outfiles = {}
for kind in corruptortypes:
    outname = out_corpus_folder + "corrupted_by." + kind
    outfiles[kind] = open(outname, "w")

processed_count = 0
# Iterate parsed sentences and test for coruptibility
print("Begining Corruption")
for parsed_sentence in sentence_gen:
    posib_trans = []
    # Test for each corruptor
    for corr in corruptortypes:
        possibility = corruptors[corr].test_possible(parsed_sentence)
        if possibility:
            posib_trans.append(corr)

    # Choose corruptor that has the fewest sentences so far
    select = None
    selectCount = float("inf")
    for possib in posib_trans:
        if corruptCount[possib] < selectCount:
            select = possib
            selectCount = corruptCount[possib]

    if select:
        # Corrupt sentence
        corruptedVersion = corruptors[select].transform(parsed_sentence)
        if corruptedVersion != -1:
            # Save corrupted sentence to corresponding file
            outfiles[select].write(corruptedVersion)
            corruptCount[select] += 1
        else:
            uncorrupted_count += 1
    else:
        uncorrupted_count += 1
    processed_count += 1
    # Progress checker, for sanity
    if processed_count % 500 == 0:
        print("{} sentences processed".format(processed_count))
# Close files
for kind in outfiles:
    outfiles[kind].close()
# Print summary to console
total = 0
for trans_type in corruptCount:
    print(trans_type + ":" + str(corruptCount[trans_type]))
    total += corruptCount[trans_type]
print("Total: {0}".format(total))
print("Incorruptible: {0}".format(uncorrupted_count))



37940
1232137


NameError: name 'transformer' is not defined